In [1]:
import numpy as np
import tensorflow as tf
import random

SEED = 1
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

import d2d_tp
import d2d
import importlib
importlib.reload(d2d_tp)
importlib.reload(d2d)
import pandas as pd
import os
import h5py

import matplotlib.pyplot as plt
import matplotlib

2022-11-16 10:39:11.131279: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 10:39:11.236367: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-16 10:39:11.266901: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-16 10:39:11.766664: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
file = "/data/fast0/datasets/Rhone_data_continuous_highpass.h5"

filt = 'Highpass'

history = {}
val_performance = {}
performance = {}

pred_DAS = {}
pred_TP = {}

true_DAS = {}
true_TP = {}

residuals_DAS = {}
residuals_TP = {}

seeds = np.arange(0, 25, 1)

In [ ]:
for SEED in seeds:

    ww = 200
    bs = 32

    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)



    linear_model_DAS, linear_model_tp, lstm_model_DAS, lstm_model_tp, dnn_model_DAS, df_all_chan, input_columns_DAS, input_columns_tp, das_data_all, f  = d2d_tp.import_data(filename = file)
    
    windows = d2d_tp.WindowGenerator(df_all_chan,
                             input_width=ww,
                             label_width=1, 
                             shift=0,
                             label_columns=['Discharge'],
                             input_columns_DAS=input_columns_DAS,
                             input_columns_tp=input_columns_tp,
                             shuffle=True,
                             batch_size=bs)
    history['LSTM_DAS_'+str(SEED)] = d2d_tp.compile_and_fit_das(lstm_model_DAS, windows)
    history['LSTM_TP_'+str(SEED)] = d2d_tp.compile_and_fit_tp(lstm_model_tp, windows)

    val_performance['LSTM_DAS_'+str(SEED)] = lstm_model_DAS.evaluate(windows.val)
    val_performance['LSTM_TP_'+str(SEED)] = lstm_model_tp.evaluate(windows.val_tp)

    performance['LSTM_DAS_'+str(SEED)] = lstm_model_DAS.evaluate(windows.test, verbose=0)
    performance['LSTM_TP_'+str(SEED)] = lstm_model_tp.evaluate(windows.test_tp, verbose=0)
    
    #Residuals

    ###DAS

    predictions_DAS = lstm_model_DAS.predict(windows.test, verbose = 0).squeeze()

    pred_test_dis_DAS = []

    for i in predictions_DAS:
        #x = (i[-1]*windows.dis_std) + windows.dis_mean #Return_sequences = True
        x = (i*windows.dis_std) + windows.dis_mean
        pred_test_dis_DAS.append(x)
    
    true_dis_DAS = []

    for i in windows.test.as_numpy_iterator():
        true_dis_DAS.append(i[1])

    true_dis_DAS = np.asarray(true_dis_DAS).squeeze()
    true_dis_DAS = (true_dis_DAS*windows.dis_std)+windows.dis_mean

    #true_dis_DAS = true_dis_DAS.reshape(true_dis_DAS.shape[0]*true_dis_DAS.shape[1])

    residual_DAS = pred_test_dis_DAS - true_dis_DAS

    pred_DAS['LSTM_DAS_'+str(SEED)] = pred_test_dis_DAS
    true_DAS['LSTM_DAS_'+str(SEED)] = true_dis_DAS
    residuals_DAS['LSTM_DAS_'+str(SEED)] = residual_DAS
    
    #Residuals

    ###Temp and Precip

    predictions_tp = lstm_model_tp.predict(windows.test_tp).squeeze()

    pred_test_dis_tp = []

    for i in predictions_tp:
        x = (i*windows.dis_std_tp) + windows.dis_mean_tp  #return_sequences = True
        #x = (i*windows.dis_std_tp) + windows.dis_mean_tp  #return_sequences = False
        pred_test_dis_tp.append(x)

    true_dis_tp = []

    for i in windows.test_tp.as_numpy_iterator():
        true_dis_tp.append(i[1])

    true_dis_tp = np.asarray(true_dis_tp).squeeze()
    true_dis_tp = (true_dis_tp*windows.dis_std_tp)+windows.dis_mean_tp

    #true_dis_tp = true_dis_tp.reshape(true_dis_tp.shape[0]*true_dis_tp.shape[1])

    residual_tp = pred_test_dis_tp - true_dis_tp
    
    pred_TP['LSTM_TP_'+str(SEED)] = pred_test_dis_tp
    true_TP['LSTM_TP_'+str(SEED)] = true_dis_tp
    residuals_TP['LSTM_TP_'+str(SEED)] = residual_tp

In [ ]:
res_DAS = []

for i in residuals_DAS.values():
    res_DAS.append(i)
    
res_TP = []

for i in residuals_TP.values():
    res_TP.append(i)
    



In [ ]:
fig, ax = plt.subplots(figsize=[10,10])
fig.patch.set_facecolor('white')

ax.hist(np.asarray(res_DAS).reshape(np.asarray(res_DAS).shape[0]*np.asarray(res_DAS).shape[1]), color='steelblue', alpha=0.5, label='DAS')
ax.hist(np.asarray(res_TP).reshape(np.asarray(res_DAS).shape[0]*np.asarray(res_DAS).shape[1]), color='brown', alpha=0.5, label = 'Temperature and Precipitation')
ax.grid()
ax.legend()
ax.set_xlabel('Residual $(m^3/s)$')
ax.set_ylabel('Count')
ax.annotate('RMSE DAS: '+str("%.2f" % rmse_DAS), (2.1, 375))
ax.annotate('RMSE Temp. and Precip.: '+str("%.2f" % rmse_TP), (2.1, 350))

plt.show()
#fig.savefig('DAS_meteo_residuals_bootstrap.pdf', dpi=1200)